[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/googlecolab/colabtools/blob/master/notebooks/colab-github-demo.ipynb)
# Docking and library searching assignment, PharmSci 175/275

## Objective
In this assignment you will perform one or more sample library searches to examine docking performance on a test set or test sets. This assignment will differ somewhat from the others in that it gives you more of a chance to explore.

## Overview
As discussed in class, docking methods are one of the fairly widely used computational methods for studying binding, and have seen significant applications in the area of virtual screening of libraries of potential ligands to filter out compounds which are unlikely to be active and hopefully increase the success rate of finding active compounds via experiments. These methods, while fairly fast, are still typically require a few seconds per compound or more, and thus often are unsuited for filtering large libraries of millions of compounds. In such cases, even faster methods such as LINGO searches may be used for pre-filtering.

Here, I am providing you with sample scripts to perform a test LINGO search to filter a library of several million compounds down to a smaller number which have similarity to a known ligand, then to dock these to a target receptor and look at enrichment of active compounds. The test case I have provided is binding of various small molecules to the LEDGF site of HIV-1 integrase, as seen in the 3D structure/shape assignment and as described in the SAMPL4 paper referenced below.

In this exercise, you will work through the example I provide, then perform tests of your own on at least three different data sets to see how well docking and LINGO searches will perform for you. You should also examine different scoring functions (within OpenEye’s docking toolkit). Think of this assignment as a bit of a sandbox - you are given some tools and a chance to play around. It is likely that the ideas you generate will help me make this assignment more successful the next time around, and any feedback is appreciated.

# Your assignment

Here, you will start off by doing a couple example activities so you can see how to use the OpenEye tools to do LINGO searches and docking. Once you’ve done these, then you will have the opportunity to branch out, try other things, and see what will work best on the system(s) you are looking at.

As background for this, I suggest you read, if you have not already done so, the paper on the [SAMPL4 HIV integrase challenge](http://dx.doi.org/10.1007/s10822-014-9723-5), as well as the [DUD paper](http://dx.doi.org/10.1021/jm0608356); the later [DUD-E paper](http://dx.doi.org/10.1021/jm300687e) may also be helpful.

Here, your warm-up work will relate to the HIV-1 integrase system studied in the SAMPL4 challenge (and seen in the 3D structure/shape assignment).

## Warm-up Part 1: Lingo search and other prep

Here, your task is to do a simple test of the Lingo search method by downloading a subset of the ZINC database, hiding some known integrase inhibitors in it, and testing the ability of the LINGO search method to enrich the database for known HIV-1 integrase inhibitors.

### First get some compounds to work with

As a starting point, let's get some compounds to work with. Visit the ZINC15 database (zinc15.docking.org), and click through on "Tranches" (subsets) towards the bottom left. This allows you to select all available compounds or subsets by purchaseability, molecular weight, calculated logP, etc. (You can toggle on and off rows and columns by clicking the top or left columns or rows), and filter by reaction likelihood (top menu) or wait time (top menu) and other properties. Select some subset of compounds (600 million is probably too many; let's keep it to a couple million or less) using these options. 

Once you've selected a subset (I filtered to get "clean" "in stock" compounds between 200-400 Daltons in some of the middle logP ranges) click the download buttom at the top right to pop up a "Download Tranches" menu. **Don't actually download**. Just copy the listed tranche codes; we can use those here to make your life easier.

Now paste your tranche codes into the box below as a string called `tranches`, like this:

## Installing Packages

***If you are running this on Google Colab, please add the installation blocks from the [getting started notebook](https://github.com/MobleyLab/drug-computing/blob/master/uci-pharmsci/Getting_Started.ipynb) or [condacolab](https://github.com/aakankschit/drug-computing/blob/master/uci-pharmsci/Getting_Started_condacolab.ipynb) here and then execute the code below***

In [ ]:
# Tranches. Replace these with your selected tranches
tranches = "ABAA ABAB ABBA ABBB ABCA ABCB BBAA BBAB BBBA BBBB BBCA BBCB CBAA CBAB CBBA CBBB CBCA CBCB DBAA DBAB DBBA DBBB DBCA DBCB EBAA EBAB EBBA EBBB EBCA EBCB FBAA FBAB FBBA FBBB FBCA FBCB ACAA ACAB ACBA ACBB ACCA ACCB BCAA BCAB BCBA BCBB BCCA BCCB CCAA CCAB CCBA CCBB CCCA CCCB DCAA DCAB DCBA DCBB DCCA DCCB ECAA ECAB ECBA ECBB ECCA ECCB FCAA FCAB FCBA FCBB FCCA FCCB ADAA ADAB ADBA ADBB ADCA ADCB BDAA BDAB BDBA BDBB BDCA BDCB CDAA CDAB CDBA CDBB CDCA CDCB DDAA DDAB DDBA DDBB DDCA DDCB EDAA EDAB EDBA EDBB EDCA EDCB FDAA FDAB FDBA FDBB FDCA FDCB AEAA AEAB AEBA AEBB AECA AECB BEAA BEAB BEBA BEBB BECA BECB CEAA CEAB CEBA CEBB CECA CECB DEAA DEAB DEBA DEBB DECA DECB EEAA EEAB EEBA EEBB EECA EECB FEAA FEAB FEBA FEBB FECA FECB AFAA AFAB AFBA AFBB AFCA AFCB BFAA BFAB BFBA BFBB BFCA BFCB CFAA CFAB CFBA CFBB CFCA CFCB DFAA DFAB DFBA DFBB DFCA DFCB EFAA EFAB EFBA EFBB EFCA EFCB FFAA FFAB FFBA FFBB FFCA FFCB AGAA AGAB AGBA AGBB AGCA AGCB BGAA BGAB BGBA BGBB BGCA BGCB CGAA CGAB CGBA CGBB CGCA CGCB DGAA DGAB DGBA DGBB DGCA DGCB EGAA EGAB EGBA EGBB EGCA EGCB FGAA FGAB FGBA FGBB FGCA FGCB"

Now we'll use the code below (using Python's `urllib` module) to download the files from these tranches and link them together into one big text file for further processing (so we don't have to download again if we need them again), and also save them to the list `zinc_compounds`. 

The file may be large. (e.g. I downloaded about 4.5 million compounds and ended up with a roughly 260 MB file; this could be reduced using compression but it's a lot of data regardless).

If you wanted to do an even larger number of compounds, you would probably want to process one tranche at a time rather than grouping them all up into one large set like I'm doing here. 

**You should just need to run this code**:

In [ ]:
# Split up the string of tranches into a list
tranches_elements = tranches.split()

# Set up an output file and list for storing results
filename = 'zinc_compounds.smi'
out_file = open(filename, 'w')
zinc_compounds = []

# Loop over SMILES string files for desired tranches 
import urllib
for t_id in tranches_elements:
    prefix = t_id[0:2]
    # Retrieve file-like object for iterating over contents of this one
    contents = urllib.request.urlopen('http://files.docking.org/2D/' + prefix + '/'+ t_id +'.smi')
    # Loop over each line, decode from bytes format to string, write to output. Skip the first line of each file
    # which is a header line
    for line in contents:
        if line.split()[0].decode() != 'smiles':
            # Replace some special characters at the same time
            out_file.write(line.decode().replace('\n',''))
            zinc_compounds.append(line.decode().replace('\r\n',''))
            
out_file.close()

### Now retrieve the SAMPL4 compounds from a file

Next, you need to load the SAMPL4 compounds and insert them in your set. These are present in this directory as `SAMPL4_smiles.smi`. Both the ZINC data and the SAMPL4 data are in a format where the SMILES string is the first entry on each line and the compound ID or name is the second entry.

**Insert code below to read in the SAMPL4 compounds (stored in `sampl4_compounds`) and then make a new list called `all_compounds` which contains the ZINC compounds plus the SAMPL4 compounds**

In [ ]:
# Your code goes here

### Let's also get a known integrase ligand for use as reference

`3NF8_ligand.pdb` in this directory contains a known HIV integrase ligand (from the 3NF8 structure in the PDB) which can be used as a query for trying to find other, similar integrase ligands, as we will do here. Using Python code or a 2D viewer, create a SMILES string for this ligand (recommended option: Use the OpenEye toolkits, read it in as an OEMol, and create an isomeric SMILES string for it) and save it into a plain text file named something like `query.smi`; this will serve as the query molecule for our LINGO search. You can also save it to a variable like `querymolecule_smiles` in this notebook.

**Your code for this goes here (if you use code) and make sure you end up with a `querymolecule_smiles` variable:**

In [ ]:
querymolecule_smiles = 'CC[C@@H](C)NC(=O)c1ccccc1C[N@@H+](CC=C)Cc2ccc3c(c2C(=O)[O-])OC[C@@H](O3)CCC(=O)[O-]'

### Update the code below to process your `all_compounds` list and filter it down to 10k compounds

Next, you'll process your set of compounds from ZINC (plus the known ligands you inserted) and compare to the reference molecule in `querymolecule_smiles`using the code below.

The code below currently takes in a `combined.smi` input file; you'll need to modify it to use your input (an `all_compounds` list).

In [ ]:
from openeye.oechem import *
import pickle

#INFILE: Input file containing SMILES strings and titles of the molecules you want to examine
#infile = 'combined.smi'
infile = 'zinc_compounds.smi'

#Query molecule: This assumes you already have `querymolecule_smiles` containing the SMILES string for your query molecule
#Load reference molecule
qmol = OEGraphMol()
OEParseSmiles( qmol, querymolecule_smiles )

#Initialize lingo comparison
lingo = OELingoSim( qmol )

#########################################################################
#Load other molecules and do the LINGO search to compute similarity scores
#########################################################################

file = open( infile, 'r') #Open input file for reading
text = file.readlines() #Read contents and close
file.close()

#Initialize storage for results
sims_by_name = {}
smiles_by_name = {}

#Loop over the text we read in and do similarity comparison
for (idx,line) in enumerate(text):

    #Every 1000 molecules, print an update on progress.
    if idx%1000==0:
        print("%s/%s" % (idx, len(text) ))

    #Extract data - split the line up into components
    tmp = line.split()

    #Initialize new, empty molecule
    mol = OEGraphMol()
    #Read SMILES into molecule from first entry on line read from file
    OEParseSmiles( mol, tmp[0] )
    
        #DO similarity comparison via lingo
    sim = lingo.Similarity( mol )

    #Load and score smiles string and name for future reference.
    smi = tmp[0]
    name = tmp[1]
    sims_by_name[ name ] = sim
    smiles_by_name[name] = smi

#########################################################################
#Process results, print out info, store results.
#########################################################################

#Get lists of similarities and compound names.
similarities = list(sims_by_name.values())
names = list(sims_by_name.keys())
print(len(names))

#Do a sort of the names by similarity score, highest to lowest.
names = sorted(names, key=lambda name: -sims_by_name[name])

#Print out maximum similarity score
print("Max similarity: %.3f" % max(similarities))

#Save top 10,000 most similar compounds (the full set is probably too large to save again) to a pickle file in case we want to re-load them to do anything else with them.
saveNum = 10000
smiles = [ smiles_by_name[name] for name in names[0:saveNum] ]
file = open('match_names_and_smiles.pickle', 'wb')
pickle.dump((names, smiles), file)
file.close()

#Save 1,000 most similar componds (again, full set is too large) to a plain text file, AND print out info on them..
file = open('match_names_and_smiles.txt', 'w')
print("Molecules, most similar to least similar:")
for n in range(saveNum):
    #Store top (saveNum) most similar molecules.
    name = names[n]
    print('%s \t %s' % (name, sims_by_name[name]))
    file.write('%s\t %s \t %s\n' % ( smiles_by_name[name], names[n], sims_by_name[name]))
file.close()

### Check how well your Lingo search did at finding the known active compounds

The script above writes the top 10,000 matches to a pickle file and a text file. Check how many of the known active compounds (from `sampl4_compounds`) are in the top 10,000. Specifically, write a python script which:
- Takes the top 10,000 Lingo matches
- Checks their names against the actives in `sampl4_compounds`
- Calcluate the enrichment factor for this 10,000 compounds (the number of actives you actually found, divided by the number you would expect to find if you were guessing randomly and the Lingo search were doing no good (so that active compounds were divided randomly throughout the full set).

You may wish to also use `oenotebook` to depict some of your top compounds and see how they compare to the actual actives. 

Ask for help if these tasks seem challenging. 

**Your python code for this should go here**:

## Warm-up Part 2: Docking

In this section, your starting point is rather similar to that in the section above — but this time, since docking is a lot slower than a LINGO search, let’s just see how simple docking can do at picking out SAMPL4 active compounds from a larger set of nonbinders. This directory contains a file called `Maybridge_nonbinders.smi`, which is non-binders from a set of fragments Tom Peat and his collaborators experimentally screened initially in SAMPL4 (which led to the lead series found in `SAMPL4_smiles.smi`). This directory also contains `3NF8_prepped.pdb`, the receptor, and `3NF8_ligand.pdb`, a reference ligand from that structure.


Also get a copy of `dock_example.py` from there, a docking tool based on an example from the OpenEye documentation, and `get_scores_from_sdf.py`, a rough Python script for analyzing the output and making an enrichment plot.

### Read in the relevant data

Read in the `Maybridge_nonbinders.smi` and `SAMPL4_smiles.smi` into a single list of lines similary to what was done above, so you have a list of compounds to dock here (we'll be docking both active and inactive compounds and seeing how well we can do at recognizing the actives). Store these into separate lists, then make a combined list of both that's called `all_docking_compounds`.

**Insert your code for this here**:

### Now dock using the compounds using the code below.

The code below will use the `Chemgauss4` scoring function from the OpenEye toolkits to dock your compounds to the target. You should try it out; you may also want to revisit it later and consider testing alternate scoring functions [described in the OpenEye documentation](https://docs.eyesopen.com/toolkits/python/dockingtk/docking.html) such as the Hybrid2, PLP, or Chemscore scoring functions. `Hybrid2` is particularly interesting as it is a combination of docking and shape comparison and could perform particularly well here since we have both the receptor AND a known ligand we can use. Comparing performance of these methods would be a useful outcome of this exercise.

This draws on code you saw in the docking sandbox.

#### Prep your receptor for docking

In [ ]:
from openeye import oedocking
from openeye import oeomega
from openeye import oechem

# Load the HIV integrase receptor from disk
imstr = oemolistream('3NF8_prepped.pdb')
protein = oechem.OEGraphMol()
oechem.OEReadMolecule(imstr, protein)
imstr.close()

# Load a reference ligand; we'll use this to indicate where the binding site is (and for `Hybrid`, it serves as a reference ligand)
ligand = oechem.OEGraphMol()
imstr = oechem.oemolistream('3NF8_ligand.pdb')
oechem.OEReadMolecule(imstr, ligand)
imstr.close()

# Initialize the receptor for docking
receptor = oechem.OEGraphMol()
oedocking.OEMakeReceptor(receptor, protein, ligand)

#### Choose your docking scoring function

If you wish to test other scoring functions, this would be where you would change it

In [ ]:
#Set the docking method and docking resolution
# Note: Chemgauss4 is the scoring function for FRED
dock_method = oedocking.OEDockMethod_Chemgauss4
dock_resolution = oedocking.OESearchResolution_Default
sdtag = oedocking.OEDockMethodGetName( dock_method )

#Generate our OEDocking object
dock = oedocking.OEDock( dock_method, dock_resolution)

#Initialize the OEDocking by providing it the receptor
if not dock.Initialize(receptor):
    # raise an exception if the receptor cannot be initialized
    raise Exception("Unable to initialize Docking with {0}".format(self.args.receptor))

#### Define a function for docking

In [ ]:
def dock_molecule( dock: "OEDock", sdtag: str, num_poses: int, mcmol ) -> tuple:
    ''' Docks the multiconfomer molecule, with the given number of poses
        Returns a tuple of the docked molecule (dockedMol) and its score
        i.e. ( dockedMol, score )
    '''
    dockedMol = oechem.OEMol()

    #Dock the molecule into a given number of poses
    res = dock.DockMultiConformerMolecule(dockedMol, mcmol, num_poses)
    
    if res == oedocking.OEDockingReturnCode_Success:
        
        #Annotate the molecule with the score and SDTag that contains the docking method
        oedocking.OESetSDScore(dockedMol, dock, sdtag)
        dock.AnnotatePose(dockedMol)
        score = dock.ScoreLigand(dockedMol)
        oechem.OESetSDData(dockedMol, sdtag, "{}".format(score))
        return dockedMol, score
    
    else:
        # raise an exception if the docking is not successful
        raise Exception("Unable to dock ligand {0} to receptor".format( dockedMol ))

#### Run docking

Note that this assumes your ligand SMILES and names are stored in `all_docking_compounds`.

You shouldn't have to change settings here, but you can if you like; e.g. for testing purposes you might wish to dock just a subset of the compounds.

In [ ]:
omega = oeomega.OEOmega()
omega.SetStrictStereo(False) 

# Generate conformers for compounds and dock
inmols = []
usednames = []
for idx,line in enumerate(all_docking_compounds):
    tmp = line.split()
    smi = tmp[0]
    mol = oechem.OEMol()
    name = tmp[1]
    if name=='' or name==None or len(name)<3:
        #Define alternate name based on index
        name = 'mol%s smiles %s' % (idx, smi)
        print("No name found on line %s; using alternate name %s..." % (idx, name))
    if not name in usednames: #Make sure haven't already used this one
        usednames.append(name)
        oechem.OEParseSmiles(mol, smi)
        mol.SetTitle(name)
        builtOK = omega(mol)
        inmols.append(mol)
    else:
        continue

#Define how many docked poses to generate per molecule
num_poses = 2


#Open a filestream for writing the docked molecules
scores = {}
with oechem.oemolostream( 'dock-results.sdf') as ofs:

    #Loop over 3D molecules from the input filestream
    for mcmol in inmols:

        #Call our written docking function
        dockedMol, score = dock_molecule( dock, sdtag, num_poses, mcmol )
        print("{} {} score = {:.4f}".format(sdtag, dockedMol.GetTitle(), score))

        #Write docked molecules to output filestream
        oechem.OEWriteMolecule(ofs, dockedMol)
        
        # Store score
        scores[ mcmol.GetTitle()] = score


Here, `dock-results.sdf` is a key output file. This is a structure file (sdf) and will contain 3D poses of molecules docked into the receptor, along with scores. You can visualize these by loading them into PyMol or a similar viewer (e.g. Vida, VMD, Chimera, ...) along with the receptor (`3NF8_prepped.pdb`).

### Now let's do an enrichment plot

If we needed to use the scores stored to file, we would start by retrieving them from the SDF file they're stored in. But we already have them in `scores`, which is a dictionary storing scores by molecule title. So here, our first step is to load active compounds and check which ones are present among our docked compounds, at which ranks.

You should be able to just run this code:

In [ ]:
# Read active compounds
active_smiles_by_name = {}
file = open('SAMPL4_smiles.smi', 'r')
text = file.readlines()
file.close()
for line in text:
    tmp = line.split()
    active_smiles_by_name[tmp[1]] = tmp[0]

# Build list of titles sorted by score
sorted_titles = list(scores.keys())
sorted_titles.sort( key = lambda title: scores[title] )

# Count how many actives are found at which ranks
ct = 0
fnd_actives = []
for active_name in active_smiles_by_name.keys():
    if active_name in sorted_titles:
        ct += 1
        print("Active %s found in docking results at rank %s" % ( active_name, sorted_titles.index(active_name)))
        fnd_actives.append( active_name )

print("Total compounds: %s" % len(sorted_titles))

#Find number of actives
n_actives = len(fnd_actives)

The next step is to actually generate an enrichment plot. This does a really basic one, which it should display inline AND save to a PDF file:

In [ ]:
#Do an enrichment plot
%pylab inline
numfound = []
ct=0
for name in sorted_titles:
    if name in active_smiles_by_name.keys():
        ct+=1
    numfound.append(ct)
fracfound = [ ct/float(n_actives) for ct in numfound]
plot( arange(len(sorted_titles)), fracfound, 'bo')
plot( [0, len(sorted_titles)], [0, 1]) #Random line
legend(('Docking results', 'Expected for random'))
xlabel('Rank')
ylabel('Fraction of actives found')
show()
savefig( 'docking-results-enrichment.pdf')

## Sandbox: Try out some other things

For the final programming part of this assignment, you have the opportunity to try out some other things *of your choice* which relate to the topic. For example, you might consider linking the two tests above -- if you dock the results of a LINGO search, how well can docking do at further enriching the active compounds? 

You also might be interested in getting the DUD-E set's HIV integrase ligands (actives) and decoys (nonbinders) -- this provides a second set of actual integrase binders which are fairly distinct from the series represented by the 3NF8 ligand (see http://dude.docking.org/targets). You could experiment to see whether docking and/or lingo searches can recognize the DUD-E integrase ligands out of the DUD-E integrase decoys, or out of ALL non-binders from DUD-E, or even whether it can recognize DUD-E integrase ligands hidden in a subset of ZINC. Experimenting here would give you a sense of how the performance of these methods depends on the dataset you are using.

You could likewise take any of the other ligand+decoy sets from DUD-E and evaluate docking and/or LINGO performance on those. 

You are free to try a variety of things, but **your final submission should include your results on at least three different datasets, and you should look at several different scoring functions. You should provide enrichment plots for every different system and scoring function examined.** Unlike some of our other assignments, relatively minimal coding is required here - mostly you’re just running things I’ve provided for you - so you should take some more time to examine the results. 

## Consider some questions

- Why does LINGO do so well here, at least in your initial test above? (Hint: Note who produced the 3NF8 structure by looking it up on the PDB. Is this part of the same series as the SAMPL4 ligands? Do you find that typical ZINC compounds bear a strong resemblance to these, or not?)
- You might be interested in looking at physical properties (size, number of rotatable bonds, polarity, etc.) of the LINGO matches to see how they compare to the SAMPL4 ligands; if so, feel free to ask if you need any help figuring out how to calculate those properties using the OpenEye tools.
- Which of the docking scoring functions you tried performs best? Do any of them perform better than random in the second example above?
- If you try docking on the results of a LINGO search — does it do any further enrichment? How does this depend on the database you dock? If you are docking compounds from ZINC, can you be sure the enrichment (or lack thereof) is real? (Hint: Do you know the compounds from ZINC are inactive?)
- If you compare results of a single method (i.e. docking) on several different databases or targets, comment on how the contents of the database, or the target you are working on, impacts performance. If possible, speculate as to why.

## Write a brief report below (or separately) for submission

Submit a brief report explaining what you did along with key plots you generated, either in this notebook or as a separate file uploaded to Canvas. Explain what you think your results mean, and answer any of the above questions you are able to. The key idea here is really to do three things:
- Explain what you did and show me the results
- Tell me what you learned and what you thought was interesting. This should include commenting on how much enrichment depends on the data set you examine and what scoring function you use. 
- Point out anything which is particularly helpful in understanding strengths and weaknesses of docking and LINGO and how they might work together.
- Also attach all enrichment plots you are reporting on, including the one you were assigned in Warm-Up Part 2